In [ ]:
%%bigquery 

with wyszukiwanie as (SELECT
    date,
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile' else 'desktop' end device,
    count(distinct concat(fullvisitorid, cast(visitstarttime as string))) sesje,
    count(distinct fullvisitorid) uzytkownicy,

FROM
    `empik-ga360.304756.ga_sessions_*`,
    UNNEST(hits) AS hits 
WHERE
    REGEXP_CONTAINS(page.pagePath, r'(.*q=.*)')  = TRUE
    AND page.searchKeyword IS NOT NULL
    and _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
group by 1,2,3,4,5),

brak_wyszukiwania as (SELECT
    date,
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile' else 'desktop' end device,
    count(distinct concat(fullvisitorid, cast(visitstarttime as string))) sesje,
    count(distinct fullvisitorid) uzytkownicy,

FROM
    `empik-ga360.304756.ga_sessions_*`,
    UNNEST(hits) AS hits 
WHERE
    REGEXP_CONTAINS(page.pagePath, r'(.*q=.*)') = FALSE
    AND page.searchKeyword IS NULL
    and _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
group by 1,2,3,4,5)

select w.date, w.true_date, w.week_of_year, w.month_of_year, w.device, w.sesje s_wyszukiwanie, w.uzytkownicy u_wyszukiwanie, b_w.sesje s_brak_wyszukiwan, b_w.uzytkownicy u_brak_wyszukiwan
from wyszukiwanie w left join brak_wyszukiwania b_w on w.date=b_w.date and w.device=b_w.device

In [ ]:
%%bigquery 

  with search as (SELECT
    date,
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile' else 'desktop' end device,
    concat(fullvisitorid, cast(visitstarttime as string)) id,
    count(distinct concat(fullvisitorid, cast(visitstarttime as string))) s_search,
    count(distinct fullvisitorid) u_search,
  FROM
    `empik-ga360.304756.ga_sessions_*`,
    UNNEST(hits) AS hits 
  WHERE
    REGEXP_CONTAINS(page.pagePath, r'(.*q=.*)')  = TRUE
    AND page.searchKeyword IS NOT NULL
    -- _table_suffix between '20190101' and '20211208'
    and _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
  GROUP BY
    1,2,3,4,5,6),

purchase as (select 
    date,
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    case when device.deviceCategory like 'mobile' then 'mobile' else 'desktop' end device,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    concat(fullvisitorid, cast(visitstarttime as string)) id,
    count(distinct concat(fullvisitorid, cast(visitstarttime as string))) s_purchase,
    count(distinct fullvisitorid) u_purchase,
    sum(hits.eventInfo.eventValue) purchase_value
FROM `empik-ga360.304756.ga_sessions_*`, unnest(hits) as hits
WHERE
    -- _table_suffix between '20200101' and '20211209'
    _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
    and eventInfo.eventCategory = 'Ecommerce'
    and eventInfo.eventAction = 'Purchase'
group by 1,2,3,4,5,6)

select s.true_date, s.week_of_year, s.month_of_year, s.device, sum(s.s_search) s_search, sum(s.u_search) u_search, sum(p.s_purchase) s_purchase, sum(p.u_purchase) u_purchase
from search s left join purchase p on s.date=p.date and s.device=p.device and s.id=p.id
group by 1,2,3,4

In [ ]:
%%bigquery 

with search as (SELECT
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile' else 'desktop' end device,
    concat(fullvisitorid, cast(visitstarttime as string)) id,
    count(distinct fullvisitorid) users,
  FROM
    `empik-ga360.304756.ga_sessions_*`,
    UNNEST(hits) AS hits 
  WHERE
    REGEXP_CONTAINS(page.pagePath, r'(.*q=.*)|(.*fromSearchQuery=.*)|(.*rq=.*)|(.*nrq=.*)|(.*qa=.*)')  = TRUE
    AND page.searchKeyword IS NOT NULL
    -- and _table_suffix between '20190501' and '20211224'
    and _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
  GROUP BY
    1,2,3,4,5),

purchase as (
select 
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile' else 'desktop' end device,
    concat(fullvisitorid, cast(visitstarttime as string)) id,
    sum(totals.totalTransactionRevenue/1000000) as all_revenue, 
    sum(totals.transactions) all_transactions
FROM `empik-ga360.304756.ga_sessions_*`
WHERE
    -- _table_suffix between '20190501' and '20211224'
    _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
group by 1,2,3,4,5)

select s.true_date, s.week_of_year, s.month_of_year, s.device, sum(s.users) users, sum(p.all_revenue) all_revenue, sum(p.all_transactions) all_transactions
from search s left join purchase p on s.true_date=p.true_date and s.device=p.device and s.id=p.id
group by 1,2,3,4 
order by 1 desc

,true_date,week_of_year,month_of_year,device,users,all_revenue,all_transactions
0,2021-12-24,202151,202112,desktop,19343,181948.30,1926
1,2021-12-24,202151,202112,mobile,42692,134938.12,1562
